在Part 1中我们提到创建工具需要构建一个schema和python函数给到model进行工具的绑定，这是最底层的实现方式，LangChain给我们进行了封装，可以通过以下方式快速创建一个供Agent调用的工具：
- `@tool`装饰器是LangChain中最简单、最直观的工具创建方式。它通过装饰器语法将普通Python函数转换为Agent可调用的工具，适合快速原型开发和简单工具实现
- `StructuredTool.from_function()`方法提供了更强大的工具创建能力，支持完整的参数校验和异步执行，适合生产环境使用。
- 通过继承`StructuredTool`类创建工具提供了最大的灵活性和控制力，适合复杂业务逻辑和状态管理需求。

| **特性** | **@tool装饰器** | **StructuredTool.from_function()** | **继承StructuredTool** |
| :---: | :---: | :---: | :---: |
| **代码简洁度** | ⭐⭐⭐⭐⭐（极简） | ⭐⭐⭐⭐（简洁） | ⭐⭐（较繁琐） |
| **参数控制** | 自动推断，弱控制 | 支持`args_schema`，强校验 | 完全自定义 Schema |
| **异步支持** | ❌（需单独定义 async 函数） | ✅（通过`coroutine`参数） | ✅（实现`_arun`方法） |
| **元数据定制** | 有限（name, description） | 中等（name, description, return_direct） | 完全定制（所有属性） |
| **适用场景** | 快速原型、简单工具 | 生产环境、需要参数校验的场景 | 复杂业务逻辑、状态管理 |
| **类型提示** | 依赖函数签名 | 结合 Pydantic 强类型 | 完整的 Pydantic 集成 |

---
| **方法** | **核心优势** | **最佳使用场景** |
| :---: | :---: | :---: |
| **@tool装饰器** | 代码简洁，一行装饰器即可完成工具注册 | 快速原型验证、简单工具实现、开发测试阶段 |
| **StructuredTool.from_function()** | 支持完整的参数校验和异步执行，适合生产环境 | 生产环境工具开发、需要严格参数校验的场景、异步操作需求 |
| **继承StructuredTool** | 最大的灵活性和控制力，支持状态管理 | 企业级复杂工具开发、需要状态管理的工具、复杂业务逻辑封装 |

# @tool的基本用法

In [ ]:
# 导入装饰器
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """在客户数据库中搜索与查询条件匹配的记录。

    Args:
        query: 要查找的搜索关键词
        limit: 返回结果的最大数量
    """
    return f"为 '{query}' 找到 {limit} 个结果"

最佳实践：
1. 提供类型提示，该提示定义了工具的输入结构。
2. 提供文档字符串，并且信息应明确且简洁，以帮助模型理解工具的用途。

## @tool的参数

| 参数名 | 类型 | 说明 |
|--------|------|------|
| name_or_callable | str \| Callable \| None | 工具名称或可调用对象，可以是字符串标识符或函数 |
| runnable | Runnable \| None | 可选的 Runnable 对象 |
| description | str \| None | 工具的描述信息 |
| return_direct | bool | 是否直接返回结果，默认为 False |
| args_schema | ArgsSchema \| None | 参数模式，用于定义工具的输入结构 |
| infer_schema | bool | 是否自动推断输入模式，默认为 True |
| response_format | Literal["content", "content_and_artifact"] | 响应格式，可以是 "content" 或 "content_and_artifact" |
| parse_docstring | bool | 是否解析文档字符串，默认为 False |
| error_on_invalid_docstring | bool | 当文档字符串无效时是否报错，默认为 True |
| extras | dict[str, Any] \| None | 额外的元数据字典 |

下面我们介绍常用的几个：工具名称、工具描述和args_schema

### 工具名称

工具名称有别于函数名，是模型与工具之间通信的关键标识符，它直接影响 Agent 能否正确调用工具，应遵循唯一名称 + 功能的“语义标签”的定义原则。

In [ ]:
@tool(name_or_callable="web_search")  # 工具名称，只有工具名称一个参数的话，可以省略关键字name_or_callable
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

### 工具描述

In [ ]:
@tool(name_or_callable="web_search", description="该工具用于根据用户提供的关键词，在网络上检索最新或相关的信息。智能体可在需要实时数据、事实核查或补充知识时调用此工具")  # 新增工具描述
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

description和函数docstring的区别：

<div align=center><img src="https://techinsight.com.cn/course/3707820c-06a6-4e5e-8615-6b0aaea1b699.png" width=80%></div>

### args_schema

函数的参数是给python程序使用的，大模型是看不到需要这些参数的，因此需要定义args_schema来告诉大模型工具函数接收什么类型的参数。LangChain支持使用Pydantic模型或JSON Schema来定义复杂的args_schema。

使用Pydantic模型：

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

# 定义天气查询的输入模型
class WeatherInput(BaseModel):
    """天气查询的输入参数。"""
    location: str = Field(description="城市名称或坐标")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="温度单位偏好"
    )
    include_forecast: bool = Field(
        default=False,
        description="是否包含5天预报"
    )

# 使用自定义输入模型定义工具
@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """获取当前天气及可选的预报信息。"""
    temp = 22 if units == "celsius" else 72
    result = f"当前 {location} 的天气：{temp} 摄氏度"
    if include_forecast:
        result += "\n未来 5 天预报：晴天"
    return result

使用JSON Schema：

In [ ]:
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """获取当前天气及可选的预报信息。"""
    temp = 22 if units == "celsius" else 72
    result = f"当前 {location} 的天气：{temp} 摄氏度"
    if include_forecast:
        result += "\n未来 5 天预报：晴天"
    return result

如果不指定args_schema，默认会根据函数的参数类型和注释来生成pydantic类型的args_schema。

In [10]:
# 导入装饰器
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """在客户数据库中搜索与查询条件匹配的记录。

    Args:
        query: 要查找的搜索关键词
        limit: 返回结果的最大数量
    """
    return f"为 '{query}' 找到 {limit} 个结果"

print(search_database)

name='search_database' description='在客户数据库中搜索与查询条件匹配的记录。\n\n    Args:\n        query: 要查找的搜索关键词\n        limit: 返回结果的最大数量' args_schema=<class 'langchain_core.utils.pydantic.search_database'> func=<function search_database at 0x106bd3420>


# StructuredTool.from_function

In [1]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool


class DivideInput(BaseModel):
    """除法工具输入参数"""
    dividend: float = Field(description="被除数")
    divisor: float = Field(description="除数，不能为零")

def divide(dividend: float, divisor: float) -> float:
    """执行除法运算，支持浮点数"""
    if divisor == 0:
        raise ValueError("除数不能为零")
    return dividend / divisor

# 2. 创建带参数校验的工具
division_tool = StructuredTool.from_function(
    func=divide,
    name="DivisionTool",
    description="安全执行除法运算，自动处理除零错误",
    args_schema=DivideInput,  # 显式指定参数模式
    return_direct=False,  # 是否直接返回工具结果（不经过 LLM 再次处理）
)

# 3. 测试参数校验（触发 Pydantic 验证）
try:
    division_tool.invoke({"a": 10, "b": 2})  # 错误：参数名不匹配
except Exception as e:
    print(f"参数校验失败：{e}")

# 4. 正确调用
result = division_tool.invoke({"dividend": 10, "divisor": 2})
print(f"除法结果：{result}")

参数校验失败：2 validation errors for DivideInput
dividend
  Field required [type=missing, input_value={'a': 10, 'b': 2}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
divisor
  Field required [type=missing, input_value={'a': 10, 'b': 2}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
除法结果：5.0


## coroutine参数

通过源码可以知道，coroutine参数用于创建异步工具。

In [ ]:
from langchain_core.tools import StructuredTool

# 同时提供同步和异步函数，优先使用异步函数
def sync_add(a: int, b: int) -> int:
    """同步加法"""
    return a + b

async def async_add(a: int, b: int) -> int:
    """异步加法"""
    return a + b

tool = StructuredTool.from_function(func=sync_add, coroutine=async_add)

# 同步调用
result = tool.invoke({"a": 1, "b": 2})
print("同步调用结果:", result)  

# 异步调用（jupyter 环境）
result = await tool.ainvoke({"a": 1, "b": 2})
print("异步调用结果:", result)  

# 异步调用（非 jupyter 环境）
import asyncio

async def main():
    result = await tool.ainvoke({"a": 1, "b": 2})  # 使用 ainvoke + await
    print(f"异步调用结果: {result}")  

asyncio.run(main())

# 继承StructuredTool

In [ ]:
import os
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool
from typing import Type

# 1. 定义包含业务逻辑的工具
class OrderQueryInput(BaseModel):
    """订单查询参数"""
    order_id: str = Field(description="订单编号，格式：ORD-2024-XXXX")
    include_details: bool = Field(default=False, description="是否包含商品明细")

class OrderQueryTool(StructuredTool):
    """订单查询工具"""
    name: str = "query_order"
    description: str = "查询电商平台订单状态和物流信息"
    args_schema: Type[BaseModel] = OrderQueryInput
    return_direct: bool = False

    def _run(self, order_id: str, include_details: bool = False) -> dict:
        # 模拟数据库查询
        order_db = {
            "ORD-2024-1234": {"status": "已发货", "express": "顺丰", "amount": 299},
            "ORD-2024-5678": {"status": "待付款", "express": "", "amount": 149},
        }

        # 处理查询逻辑
        if order_id not in order_db:
            return {"error": f"订单 {order_id} 不存在"}
        result = order_db[order_id]

        # 处理包含明细的情况
        if include_details:
            result["items"] = ["商品A × 2", "商品B × 1"]

        return result

# 2. 初始化模型
model = init_chat_model(
    model="deepseek-chat",
    temperature=0
)

# 3. 创建 ReAct Agent（自动处理工具调用）
agent = create_agent(
    model=model,
    tools=[OrderQueryTool()],  # 直接传入 StructuredTool 实例
    system_prompt="你是一个电商客服助手，使用工具查询订单信息，回答要友好且准确",
    checkpointer=InMemorySaver()
)

# 4. 执行并观察 ReAct 过程
async def run_agent():
    config = {"configurable": {"thread_id": "customer_001"},"recursion_limit": 15} # 最大 15 次迭代

    query = "请帮我查订单 ORD-2024-1234 的详细状态，包括商品明细"

    async for step in agent.astream(
        {"messages": [{"role": "user", "content": query}]},
        config=config,
        stream_mode="updates"  # 流式输出模式，返回每一步的完整状态
    ):
        # stream_mode="values"用这个打印
        # message = step["messages"][-1]
        # message.pretty_print()
        print(step)
        print("-" * 50)

# 运行
await run_agent()

{'model': {'messages': [AIMessage(content='我来帮您查询订单 ORD-2024-1234 的详细状态，包括商品明细。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 379, 'total_tokens': 467, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 59}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'f60c8f19-8c4c-4394-b096-1e7627b66fa2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ca36a-5987-7341-9426-fc894ac6cb9b-0', tool_calls=[{'name': 'query_order', 'args': {'order_id': 'ORD-2024-1234', 'include_details': True}, 'id': 'call_00_o1tPjpiY1bMZ3i92BayEULsB', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 379, 'output_tokens': 88, 'total_tokens': 467, 'input_token_details': {'cache_read': 320}, 'output_token_details': {}

在异步调用会进入StructuredTool._arun()方法，默认情况下，会检测是否存在coroutine方法，如果存在，就优先使用coroutine定义的方法，否则为了防止同步阻塞代码（Sync code）卡住主异步事件循环（Event Loop），框架会自动将这个同步任务放到一个后台线程池中去执行。

# 查看运行流程

这里提供一个查看Agent运行流程的方法，供大家后续调试使用。

In [15]:
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.agents import create_agent

# 引入 UI 库
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich.markdown import Markdown

# 初始化控制台
console = Console()

# --- 第一步：定义工具 (和以前一样，这是 Core 标准) ---
# 定义天气查询工具
@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息。"""
    weather_data = {
        "北京": "晴朗，气温25°C",
        "上海": "多云，气温28°C",
        "广州": "小雨，气温30°C"
    }
    return f"{city}的天气是：{weather_data.get(city, '未知')}"

# 定义数学计算工具
@tool
def add(a: float, b: float) -> float:
    """计算两个数的和"""
    return a + b

tools = [get_weather, add]

# --- 第二步：初始化模型 ---
model = init_chat_model(model="deepseek-chat", temperature=0)

# --- 第三步：构建图 (使用 prebuilt 的 ReAct Agent) ---
graph = create_agent(model, tools=tools)

# --- 第四步：编写可视化流式运行器 ---
def run_demo_with_visualization(user_input: str):
    print("\n" + "="*50)
    console.print(f"[bold yellow]开始任务：[/bold yellow] {user_input}")

    messages = [HumanMessage(content=user_input)]

    # graph.stream 是 LangGraph 的核心
    # 它可以让我们看到状态流转的每一步 (step-by-step)
    step_count = 1

    # values 模式会返回当前的 message 列表状态
    for event in graph.stream({"messages": messages}, stream_mode="values"):
        # 获取最新的一条消息
        current_message = event["messages"][-1]

        # 1. 如果是人类的消息 (初始状态)
        if isinstance(current_message, HumanMessage):
            continue # 跳过，这是输入

        # 2. 如果是 AI 的消息 (思考与决策)
        if isinstance(current_message, AIMessage):
            # 检查是否有工具调用
            if current_message.tool_calls:
                # 提取工具调用的细节
                for tool_call in current_message.tool_calls:
                    console.print(Panel(
                        Text(f"🤔 AI 思考决定：需要调用外部工具\n"
                             f"🔧 工具名称: {tool_call['name']}\n"
                             f"📥 输入参数: {tool_call['args']}", style="bold cyan"),
                        title=f"Step {step_count}: 决策 (Decision)",
                        border_style="cyan"
                    ))
            else:
                # 如果没有工具调用，说明是最终回复
                console.print(Panel(
                    Markdown(current_message.content),
                    title=f"Step {step_count}: 最终回复 (Final Answer)",
                    border_style="green"
                ))
            step_count += 1

        # 3. 如果是工具的消息 (观察与结果)
        if isinstance(current_message, ToolMessage):
            console.print(Panel(
                Text(f"👀 工具返回结果 (Observation):\n{current_message.content}", style="italic white"),
                title=f"Step {step_count}: 执行与观察",
                border_style="magenta"
            ))
            step_count += 1

# --- 第五步：运行演示 ---
if __name__ == "__main__":
    # 这是一个多步任务：先算乘法，再查属性
    run_demo_with_visualization("查询一下北京和上海气温，并且计算一下北京的温度比上海低多少度？")

开始任务： 查询一下北京和上海气温，并且计算一下北京的温度比上海低多少度？

╭──────────────────────────────────────────── Step 1: 决策 (Decision) ────────────────────────────────────────────╮
│ 🤔 AI 思考决定：需要调用外部工具                                                                                │
│ 🔧 工具名称: get_weather                                                                                        │
│ 📥 输入参数: {'city': '北京'}                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Step 2: 执行与观察 ───────────────────────────────────────────────╮
│ 👀 工具返回结果 (Observation):                                                                                  │
│ 北京的天气是：晴朗，气温25°C                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Step 3: 决策 (Decision) ────────────────────────────────────────────╮
│ 🤔 AI 思考决定：需要调用外部工具                                                                                │
│ 🔧 工具名称: get_weather                                                                                        │
│ 📥 输入参数: {'city': '上海'}                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Step 4: 执行与观察 ───────────────────────────────────────────────╮
│ 👀 工具返回结果 (Observation):                                                                                  │
│ 上海的天气是：多云，气温28°C                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Step 5: 决策 (Decision) ────────────────────────────────────────────╮
│ 🤔 AI 思考决定：需要调用外部工具                                                                                │
│ 🔧 工具名称: add                                                                                                │
│ 📥 输入参数: {'a': 28, 'b': -25}                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Step 6: 执行与观察 ───────────────────────────────────────────────╮
│ 👀 工具返回结果 (Observation):                                                                                  │
│ 3.0                                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Step 7: 最终回复 (Final Answer) ────────────────────────────────────────╮
│ 根据查询结果：                                                                                                  │
│                                                                                                                 │
│  • 北京：晴朗，气温25°C                                                                                         │
│  • 上海：多云，气温28°C                                                                                         │
│                                                                                                                 │
│ 北京的温度比上海低 3°C（28°C - 25°C = 3°C）。                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯